# Django的设计流程

## 一、创建一个DJango的项目

``` shell
django-admin startproject blogdemo

```

## 二、创建项目中的一个app
```shell

python manage.py startapp index
```

## 三、配置数据库
进入项目目录查找seeting文脚本修改一下代码

```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': os.path.join(BASE_DIR, 'db.sqlite3'),
    }
}


```

修改为

```python
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.mysql',
        # 数据库名称
        'NAME': 'testblog',
        'USER': 'root',
        'PASSWORD': '123456',
        'HOST': 'localhost',
        'PORT': '3306',
    }
}


```

### 在init.py文件中添加如下语句

```python
import pymysql
pymysql.install_as_MySQLdb()

```

## 四、配置app
将seeting中一下代码

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'index',
]

```

修改为

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]
```

##### 配置中文时间
```python
LANGUAGE_CODE = 'zh-Hans'

TIME_ZONE = 'Asia/Shanghai'

```

## 五、设计数据表结构

总共三个字段

#### 文章类别　tag

id | tag_name 
-|-
integer pk |  charset not null

#### 文章内容　post 

id | name | time | auther | tag
-|-|-|-|-
integer pk |  charset not null | datatime |charset | charset

#### 文章作者　auther

id | name | sex | age 
-|-|-|-
integer pk |  charset not null | bool |charset

## 编写model

```python
from django.db import models


# Create your models here.
class Tag(models.Model):
    tag_name = models.CharField(max_length=50, null=False, verbose_name="名称")

    class Meta:
        verbose_name = verbose_name_plural = '类别'

    def __str__(self):
        return self.tag_name


class Auther(models.Model):
    STATUS_NORMAL = 1
    STATUS_DELETE = 0
    STATUS_ITEMS = (
        (STATUS_NORMAL, '男性'),
        (STATUS_DELETE, '女性'),
    )
    # null = True的时候才是设置不能为空！
    auther_name = models.CharField(max_length=50, null=True, verbose_name="作者姓名")

    sex = models.PositiveIntegerField(default=STATUS_NORMAL, choices=STATUS_ITEMS, verbose_name="作者性别")

    age = models.IntegerField(verbose_name="作者年龄")

    class Meta:
        verbose_name = verbose_name_plural = '作者'

    # 这个是在admin中返回作者的创建成功后的列表
    def __str__(self):
        return self.auther_name


class Post(models.Model):
    post_name = models.CharField(max_length=50, null=True, verbose_name="名称")
    time = models.TimeField()
    auther = models.ForeignKey(Auther, verbose_name="作者姓名", on_delete=models.DO_NOTHING)
    tag = models.ForeignKey(Tag, verbose_name="文章类别", on_delete=models.DO_NOTHING)
    content = models.TextField(max_length=50, null=True, verbose_name="文章内容")

    class Meta:
        verbose_name = verbose_name_plural = '文章'

    def __str__(self):
        return self.post_name





```

##### 常用的Model的模板



```python
from django.db import models


class Tag(models.Model):
    # 常见的字符串类型的字段
    tag_name = models.CharField(max_length=50, null=False, verbose_name="名称",unique = True,help_text = "test",default = "liyuan3970")
    # 选项类型的字段设置
    STATUS_NORMAL = 1
    STATUS_DELETE = 0
    STATUS_ITEMS = (
        (STATUS_NORMAL, '男性'),
        (STATUS_DELETE, '女性'),
    )    
    sex = models.PositiveIntegerField(default=STATUS_NORMAL, choices=STATUS_ITEMS, verbose_name="作者性别")
    
    # 整数类型的字段的设置
    age = models.IntegerField(verbose_name="作者年龄")
    
    # 外键，又可以是一对一的数据结构
    auther = models.ForeignKey(Auther, verbose_name="作者姓名", on_delete=models.DO_NOTHING)
    
    # 长字符串类型的数据
    content = models.TextField(max_length=50, null=True, verbose_name="文章内容")
   
    # 时间类型的数据结构
    time = models.TimeField()
    
    # 多对多的数据结构
    tag = models.ManyToManyField(Tag, verbose_name="标签")
    
    
    # 定义数据库中表的名称，以及排列顺序
    class Meta:
        verbose_name = verbose_name_plural = '类别'
        ordering = ['-id']
        db_table = 'tag' # 通过db_table自定义数据表名
     
    # 定义后台列表中的字段名称
    def __str__(self):
        return self.tag_name







```


##### 基本的字段在后台的配置布局模板

```python
from django.contrib import admin
from .models import *
# Register your models here.

class PostAdmin(admin.ModelAdmin):
    #显示所有文章的字段
    list_display = ('post_name', 'time', 'auther','tag')
    # 对应的字段可以作为超链接进去
    list_display_links = ('auther', 'tag')
    # 自爱文章列表中可以修改的字段，和link不能重叠
    list_editable = ('time',)


    # 在文章列表右侧添加一个过滤的选项
    list_filter = ('auther',)
    # 在文章列表顶部添加一个搜索狂，方便用户查找，里面的字段是查询的关键字
    search_fields = ('post_name', 'time')

    # 这个编辑页面的基本布局
    fieldsets = (
        #组1-基本选项,包含post_name和auther两个列
        ('基本选项',{
            'fields':('post_name','auther')
        }),
        #组2-高级选项,包含tag和time和content
        ('高级选项',{
            'fields':('tag','time','content'),
            'classes':('content',)
        })
    )
admin.site.register(Auther)
admin.site.register(Post,PostAdmin)
admin.site.register(Tag)

```
##### 后台对应的字段布局形式

![后台1](src/1.png)

![后台2](src/2.png)



##### Models对应的CRUD

```python
from django.http import HttpResponse
from django.shortcuts import render
from .models import *

## 增加数据
def add_views(request):
    # 方式1:通过Entry.objects.create() 增加一条数据
    obj = Author.objects.create(
        name='吕泽Maria',
        age = 26,
        email='maria@163.com'
    )
    print(obj)
    
    #方式2:通过obj.save()增加1条数据
    obj = Author(name='莫言',age=60)
    obj.email = 'moyan@163.com'
    obj.save()
    print("注册的用户ID为:",obj.id)
    
    #方式3:通过字典创建对象再通过save()完成增加
    dic = {
        "name":"老舍",
        "age":105,
        "email":"laoshe@163.com",
        "isActive":False
    }
    obj = Author(**dic)
    obj.save()
    print("注册的用户ID为:", obj.id)
    
    # 练习
    Publisher.objects.create(
        name='北京大学出版社',
        address='北大东路250号',
        city='北京',
        country='中国',
        website='http://www.beida.com'
    )
    
    pub2 = Publisher(
        name='清华大学出版社',
        address='清华东路250号',
        city='北京',
        country='中国',
        website='http://www.tsinghua.com'
    )
    pub2.save()
    
    dicPub = {
        "name":"华北大学出版社",
        "address":"五道口250号",
        "city":"天津",
        "country":"中国",
        "website":"http://www.huabei.com"
    }
    pub3 = Publisher(**dicPub)
    pub3.save()

    Book.objects.create(title='钢铁是咋炼成的',publicate_date='1996-10-12')

    book2 = Book(
        title = '钢铁是咋炼成的',
        publicate_date = '1996-10-12'
    )
    book2.save()

    dicBook = {
        'title':'钢铁是咋炼成的',
        'publicate_date':'1996-10-12'
    }
    book3 = Book(**dicBook)
    book3.save()
    return HttpResponse("<script>alert('增加数据成功!');</script>")


# 查询数据
def query_views(request):
    #1. 查询Author.objects 的返回值以及类型
    obj = Author.objects
    print(obj)
    print("type:",type(obj))

    #2. Entry.objects.all()
    authors = Author.objects.all()
    for au in authors:
        print("ID:%d,姓名:%s,年龄:%d,邮箱:%s" % (au.id,au.name,au.age,au.email))
    print(authors)
    print("type:",type(authors))

    #3. Entry.objects.values()
    authors = Author.objects.values('name','email')
    print(authors)
    print("type",type(authors))
    return HttpResponse("<script>alert('查询成功')</script>")

def query_exer(request):
    # 1.查询出age大于等于30的Author的信息
    # authors = Author.objects.filter(age__gte=30)
    # print(authors)
    # 2.查询出所有姓莫的Author的信息
    # author = Author.objects.filter(name__startswith='莫')
    # print(author)
    # 3.查询出Email中包含an字符的所有的Author的信息
    authors = Author.objects.filter(email__contains='an')
    # 4.查询出书籍的出版时间在1990年~2000年之间的出版的书籍信息
    books = Book.objects.filter(publicate_date__year__range=(1990,2000))
    # for book in books :
    #     print('书名:%s,出版时间:%s' % (book.title,book.publicate_date))
    # 5.查询出Author中年龄大于莫言年龄的人的信息
    # Author.objects.filter(name='莫言').values('age')

    authors = Author.objects.filter(
        age__gt=Author.objects.filter(name='莫言').values('age'))
    for au in authors:
        print("姓名:%s,年龄:%d" % (au.name,au.age))
    return HttpResponse("查询成功")

# 删除数据
def delete_views(request,id):
    author = Author.objects.get(id=id)
    author.isActive = False
    author.save()
    return redirect('/06-queryall')

#  更改数据
def update_views(request):
    Author.objects.all().update(age=F('age')+10)
    return HttpResponse("更新成功")

```


## 六、编写视图配置URL

##### 分布式路由系统的配制方法

###### 根目录url的配置

```python
from django.conf.urls import url,include
from django.contrib import admin
# include 函数用来配置分布式路由
urlpatterns = [
    url(r'^admin/', admin.site.urls),
    # 对应的是每一个项目的目录名称
    url(r'^',include('index.urls')),

]
```
###### 项目目录url的配置
```python
from django.conf.urls import url
from . import views

urlpatterns = [
    url(r'^$',views.index),
    
]

```